# How to use the MLencoding class

This is a tutorial of how to use our MLencoding package to build encoding models a predict spikes. 


In [18]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import scipy.io

##### Load encoding package

In [19]:
from MLencoding import *

# 1. Data
Below we load a dataset available on CRCNS: a [Macaque M1](http://crcns.org/data-sets/movements/dream/downloading-dream) (from [Stevenston et al. 2011](http://jn.physiology.org/content/106/2/764.short)).

The data has been organized in Matlab into neat arrays for easy loading here.

We will soon want a single numpy array representing the external covariates, and a single numpy vector representing the neural response. The data array X will be of dimensions (n, p), where n is the number of time bins and p is the number of covariates, and the response y will be of dimensions (n, ) . We use pandas as an intermediate tool for data organizing, but it's really not necessary - if using your own data just wrangle it into numpy arrays of proper dimension.

#### Load data

In [3]:
m1_imported = scipy.io.loadmat('../data/m1_stevenson_2011.mat')

### 1.1 Covariates

Pull into pandas dataframe. This allows us to easily access covariates by name.

In [4]:
data = pd.DataFrame()
data['time'] =  m1_imported['time'][0]
data['handPos_x'] =  m1_imported['handPos'][0]
data['handPos_y'] =  m1_imported['handPos'][1]
data['handVel_x'] =  m1_imported['handVel'][0]
data['handVel_y'] =  m1_imported['handVel'][1]

#### Compute more covariates/features

#These will be used as the 'engineered' features for improving the GLM's performance.

data['velDir'] = np.arctan2(data['handVel_y'], data['handVel_x'])
data['cos_velDir'] = np.cos(data['velDir'])
data['sin_velDir'] = np.sin(data['velDir'])
data['speed'] = np.sqrt(data['handVel_x'].values**2+data['handVel_y'].values**2)
r = np.arctan2(data['handPos_y'], data['handPos_x'])
data['cos_PosDir'] = np.cos(r)
data['sin_PosDir'] = np.sin(r)
data['radial_Pos'] = np.sqrt(data['handPos_x'].values**2+data['handPos_y'].values**2)
data.head()

,time,handPos_x,handPos_y,handVel_x,handVel_y,velDir,cos_velDir,sin_velDir,speed,cos_PosDir,sin_PosDir,radial_Pos
0,12.591,0.002905,-0.303636,-0.011201,-0.006237,-2.633523,-0.873685,-0.486491,0.012820,0.009568,-0.999954,0.303650
1,12.641,0.002260,-0.303869,-0.010743,-0.000833,-3.064245,-0.997010,-0.077271,0.010775,0.007437,-0.999972,0.303877
2,12.691,0.002399,-0.303631,0.017680,0.012094,0.599956,0.825360,0.564606,0.021420,0.007900,-0.999969,0.303641
3,12.741,0.004010,-0.302399,0.044667,0.038700,0.713933,0.755792,0.654812,0.059100,0.013258,-0.999912,0.302426
4,12.791,0.006386,-0.300673,0.042202,0.017021,0.383375,0.927408,0.374053,0.045505,0.021233,-0.999775,0.300741


# 2. Making an encoding model

We instantiate the object like this:

In [5]:
glm_model = MLencoding(tunemodel = 'glm')

We can then train it on some data. Let's go for 3/4 of the data we have for some neuron.

In [8]:
neuron_n = 1

X = data[['handPos_x','handPos_y','handVel_x','handVel_y']].values
y = m1_imported['spikes'][neuron_n]

n_samples = X.shape[0]
threefourths = int(n_samples*3/4)

X_train = X[:threefourths,:]
y_train = y[:threefourths]


In [ ]:
# Now we train the model

glm_model.fit(X_train,y_train)

Let's predict the neural response on the training set.

In [10]:
X_test = X[threefourths:,:]
y_test = y[threefourths:]

y_hat = glm_model.predict(X_test)

How did we do? We can score this prediction with the class's internal function 'poisson_pseudoR2'.

In [11]:
# The 'null model' we measure against is the mean of the train dataset. 
y_null = np.mean(y_train)

pr2_glm = glm_model.poisson_pseudoR2(y_test, y_hat, y_null)
print(pr2_glm)

0.0625913964434


## Cross-validation

Let's now obtain the predictions and scores of 10-fold cross-validation for a GLM.

In [18]:
Y_hat, PR2s = glm_model.fit_cv(X,y, n_cv = 10, verbose = 2)

...runnning cv-fold 1 of 10
pR2:  0.0488023178838
...runnning cv-fold 2 of 10
pR2:  0.0434830590622
...runnning cv-fold 3 of 10
pR2:  0.0513488923378
...runnning cv-fold 4 of 10
pR2:  0.0521074580784
...runnning cv-fold 5 of 10
pR2:  0.0449312912574
...runnning cv-fold 6 of 10
pR2:  0.062685886475
...runnning cv-fold 7 of 10
pR2:  0.0459586387009
...runnning cv-fold 8 of 10
pR2:  0.0578141187789
...runnning cv-fold 9 of 10
pR2:  0.0523027349251
...runnning cv-fold 10 of 10
pR2:  0.0496125678667
pR2_cv: 0.050905 (+/- 0.001765)


### Other methods: neural networks, random forest, XGBoost

Using other encoding models is as simple as this:

In [16]:
nn_model = MLencoding(tunemodel='feedforward_nn')

In [17]:
Y_hat, PR2s = nn_model.fit_cv(X,y, n_cv = 10, verbose = 2)

...runnning cv-fold 1 of 10
pR2:  0.0973511536366
...runnning cv-fold 2 of 10
pR2:  0.105358255116
...runnning cv-fold 3 of 10
pR2:  0.105960770234
...runnning cv-fold 4 of 10
pR2:  0.118384046484
...runnning cv-fold 5 of 10
pR2:  0.0824833296985
...runnning cv-fold 6 of 10
pR2:  0.111601716963
...runnning cv-fold 7 of 10
pR2:  0.11344632257
...runnning cv-fold 8 of 10
pR2:  0.109179061213
...runnning cv-fold 9 of 10
pR2:  0.112530046581
...runnning cv-fold 10 of 10
pR2:  0.0907163987897
pR2_cv: 0.104701 (+/- 0.003373)


### Predicting spikes using spike or covariate history

MLencoding supports models that also use previous covariate values to predict the current spike rate. Spike history is also supported.

When you instantiate a model with the `spike_history=True` or `cov_history=True` keywords, all future calls to `fit`, `predict`, and `fit_cv` will automatically construct a new covariate matrix with additional columns. These columns represent the covariate history. This matrix is then used for fitting.

Currently, covariate history columns are raised cosine basis functions. You can define how many temporal basis you want with `n_filters`, which will span the interval [0, `max_time`]. Times are measured in milliseconds. In order to perform this calculation, the model needs to know how many milliseconds are in each time bin. (Set this with `window`).

In [6]:
xgb_history = MLencoding(tunemodel = 'xgboost',
                         cov_history = False, spike_history=True, # We can choose!
                         window = 50, #this dataset has 50ms time bins
                         n_filters = 2,
                         max_time = 250 )

In [7]:
xgb_history.fit_cv(X,y, verbose = 2, continuous_folds = True);

...runnning cv-fold 0 of 10
pR2:  0.172896381616
...runnning cv-fold 1 of 10
pR2:  0.151629755677
...runnning cv-fold 2 of 10
pR2:  0.183958679349
...runnning cv-fold 3 of 10
pR2:  0.149697611433
...runnning cv-fold 4 of 10
pR2:  0.127944114605
...runnning cv-fold 5 of 10
pR2:  0.146583568384
...runnning cv-fold 6 of 10
pR2:  0.227747587776
...runnning cv-fold 7 of 10
pR2:  0.265500709309
...runnning cv-fold 8 of 10
pR2:  0.275622248323
...runnning cv-fold 9 of 10
pR2:  0.266005528721
pR2_cv: 0.196759 (+/- 0.017011)


Here is a version that uses spike history with random folds.

In [6]:
# First we need to set n_every > max_time/window. 
xgb_history_rand = MLencoding(tunemodel = 'xgboost',
                         cov_history = False, spike_history=True,
                         window = 50, 
                         n_filters = 2,
                         max_time = 250, n_every = 6 )

xgb_history_rand.fit_cv(X,y, verbose = 2, continuous_folds = False);

...runnning cv-fold 1 of 10
pR2:  0.172660664684
...runnning cv-fold 2 of 10
pR2:  0.201177093824
...runnning cv-fold 3 of 10
pR2:  0.181089793866
...runnning cv-fold 4 of 10
pR2:  0.148885335305
...runnning cv-fold 5 of 10
pR2:  0.183087263289
...runnning cv-fold 6 of 10
pR2:  0.17288721494
...runnning cv-fold 7 of 10
pR2:  0.130874947193
...runnning cv-fold 8 of 10
pR2:  0.175744079298
...runnning cv-fold 9 of 10
pR2:  0.149755921527
...runnning cv-fold 10 of 10
pR2:  0.0676141844825
pR2_cv: 0.158378 (+/- 0.011328)


## Fitting an LSTM

There's nothing special about fitting an LSTM in our implementation. Just be sure to set `spike_history=True` and `cov_history = True`, and to use continuous CV folds.

In [6]:
lstm = MLencoding(tunemodel = 'lstm',
                         cov_history = True, spike_history=True, # We can choose!
                         window = 50, #this dataset has 50ms time bins
                         n_filters = 4,
                         max_time = 250 )

In [7]:
lstm.fit_cv(X,y, verbose = 2, continuous_folds = True);

...runnning cv-fold 0 of 10
pR2:  0.178190232035
...runnning cv-fold 1 of 10
pR2:  0.169885240103
...runnning cv-fold 2 of 10
pR2:  0.176461553019
...runnning cv-fold 3 of 10
pR2:  0.161520848555
...runnning cv-fold 4 of 10
pR2:  0.132098223238
...runnning cv-fold 5 of 10
pR2:  0.149307415463
...runnning cv-fold 6 of 10
pR2:  0.246421820011
...runnning cv-fold 7 of 10
pR2:  0.269467384959
...runnning cv-fold 8 of 10
pR2:  0.276576452182
...runnning cv-fold 9 of 10
pR2:  0.281433609311
pR2_cv: 0.204136 (+/- 0.017292)


### Getting and setting model parameters
To get the current set of parameters, we can either run:

In [18]:
nn_model.params
# or nn_model.get_params()

{'dropout': 0.5, 'l1': 0.0, 'l2': 0.0, 'n1': 1980, 'n2': 18}

We can set the parameters with the `set_params` method. This method takes a dictionary, which update the current set of parameters used.

In [ ]:
nn_model.set_params({'dropout':0.3})
nn_model.params

### Hyperparameter optimization using hyperopt

We might not want the default parameters. Here's how to set some better ones

In [16]:
from hyperopt import fmin, hp, Trials, tpe, STATUS_OK

# Makes sure these are in nn_models.params, otherwise you'll get a key error
space4rf = {
    'dropout': hp.uniform('dropout', 0., 0.6),
    'n1': hp.uniform('n1', 2,128),
    'n2': hp.uniform('n2', 1,15),
}

#object that holds iteration results
trials = Trials()

#define model
nn_model = MLencoding(tunemodel='feedforward_nn')

#function to minimize
def fnc(params):
    
    # make sure parameters are integers that need to be. 
    params['n1'] = int(params['n1'])
    params['n2'] = int(params['n2'])

    nn_model.set_params(params)
    
    # Remember that X and y have been defined above.
    Y_hat, PR2s = nn_model.fit_cv(X,y, n_cv = 5, verbose = 0)

    # return negative since hyperopt always minimizes the function
    return -np.mean(pseudo_R2)

Let's assume that our neuron #1 is a held-out neuron for parameter optimization. Let's optimize:

In [ ]:
hyperoptBest = fmin(fnc, space4rf, algo=tpe.suggest, max_evals=50, trials=trials)

# Defining your own models

The `MLencoding` class is flexible and can be used with predefined models as long as they have `fit` and `predict` methods.

Let's build a different type of neural network, for example.

In [30]:
my_model = Sequential()
my_model.add(Dense(100, input_dim=np.shape(X)[1], init='glorot_normal',
            activation='relu',))
my_model.add(Dense(1,activation='softplus'))
optim = Nadam()
my_model.compile(loss='poisson', optimizer=optim,)


In [31]:
my_enc = MLencoding(tunemodel = my_model)

In [33]:
my_enc.fit_cv(X,y,n_cv=5,verbose=2);

...runnning cv-fold 1 of 5
pR2:  -0.00401729001754
...runnning cv-fold 2 of 5
pR2:  -0.00440856722819
...runnning cv-fold 3 of 5
pR2:  -0.00344133554292
...runnning cv-fold 4 of 5
pR2:  -0.000698628352245
...runnning cv-fold 5 of 5
pR2:  -0.00209311949187
pR2_cv: -0.002932 (+/- 0.000610)


This model isn't great, but you see how it's possible. 

There are some limitations here, though. One thing I can think off the bat is that this Keras model won't work if we set `spike_history = True`, since that will change the shape of `X` and the shape of the input layer is hard-coded when we built this model. 